## Delivery Truck Packing Problem 
In North America there are over 100 standard box sizes that consumer goods are shipped in.

Optimally loading delivery trucks with packages, expecially when additional considerations such as priority shipping status, order date and weight/size limits of the trucks are considered is a difficult combinatorial optimization problem.

### Objectives:
1. Maximize the number of packages selected with priority shipping
2. Minimize the number of days the packages are in transit

### Constraints:
1. Do not exceed the maximum number of packages that can fit on the truck (100)
2. Do not exceed the maximmum weight capacity of the truck (3000 lbs)

In [1]:
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel, Binary, quicksum
import random
import numpy as np

In [26]:
cqm=ConstrainedQuadraticModel()

In [27]:
random.seed(2002)
num_packages=300
priority=[random.choice([1, 2, 3]) for i in range(num_packages)]
days_since_order=[random.choice([0, 1, 2, 3]) for i in range(num_packages)]
cost=[random.randint(1, 100) for i in range(num_packages)]
num_items=len(cost)
print(cost)

[38, 31, 48, 55, 9, 46, 71, 73, 4, 46, 86, 98, 36, 13, 78, 67, 17, 68, 7, 35, 46, 34, 46, 39, 71, 93, 15, 33, 60, 71, 22, 92, 77, 14, 91, 100, 60, 76, 91, 55, 24, 59, 83, 98, 8, 23, 28, 89, 16, 16, 89, 32, 9, 85, 40, 46, 83, 43, 23, 17, 19, 64, 100, 62, 45, 30, 97, 88, 28, 26, 53, 58, 27, 5, 78, 29, 36, 69, 66, 82, 95, 51, 11, 60, 96, 46, 1, 62, 85, 56, 58, 32, 28, 65, 21, 35, 92, 58, 20, 38, 3, 73, 80, 41, 53, 88, 45, 78, 6, 83, 63, 38, 97, 35, 10, 90, 47, 33, 16, 74, 90, 72, 23, 88, 53, 24, 17, 60, 10, 79, 74, 8, 44, 37, 37, 77, 100, 7, 18, 45, 86, 77, 74, 16, 27, 78, 92, 16, 29, 72, 45, 15, 38, 99, 28, 87, 75, 100, 47, 86, 92, 27, 72, 20, 82, 39, 19, 45, 19, 99, 39, 32, 78, 82, 46, 67, 44, 47, 58, 64, 48, 42, 13, 3, 30, 83, 59, 15, 86, 99, 3, 76, 88, 7, 87, 69, 42, 80, 38, 46, 55, 17, 94, 32, 54, 35, 34, 24, 50, 27, 27, 65, 28, 9, 72, 62, 80, 74, 31, 42, 100, 55, 73, 74, 10, 17, 3, 23, 64, 19, 61, 78, 61, 42, 100, 29, 33, 75, 40, 86, 76, 81, 41, 41, 49, 28, 31, 93, 55, 91, 29, 98, 3

In [28]:
max_weight=3000
max_parcels=100
obj_weight_priority=1.0
obj_weight_days=1

In [29]:
bin_variables=[Binary(i) for i in range(num_items)]

#### Objective 1: Maximize priority shipping $ min(-\sum_{i=0}^{N}p_i\cdot x_i) $

In [30]:
obj1=-obj_weight_priority*quicksum(priority[i]*bin_variables[i] for i in range(num_items))

#### Objective 2: Minimize wait time $ min(-\sum_{i=0}^{N}d_i\cdot xi) $

In [31]:
obj2=-obj_weight_days*quicksum(days_since_order[i]*bin_variables[i] for i in range(num_items))

In [32]:
cqm.set_objective(obj1+obj2)

#### Constraint 1: Maximum parcels $ \sum_{i=0}^{N}x_i=P $

In [33]:
cqm.add_constraint(quicksum(cost[i]*bin_variables[i] for i in range(num_items)) <= max_weight, label='max_capacity')

'max_capacity'

#### Constraint 2: Maximum capacity $ \sum_{i=0}^{N}w_i\cdot x_i\le W $

In [34]:
cqm.add_constraint(quicksum(bin_variables[i] for i in range(num_items)) <= max_parcels, label='max_parcels')

'max_parcels'

In [35]:
cqm_sampler=LeapHybridCQMSampler()
samplest=cqm_sampler.sample_cqm(cqm, label='Truck Packing Demo')

In [36]:
print(samplest.info)

{'constraint_labels': ['max_capacity', 'max_parcels'], 'qpu_access_time': 31851, 'charge_time': 4789668, 'run_time': 4789668, 'problem_id': 'd9d66307-0c1c-443b-9424-3187435bf70c', 'problem_label': 'Truck Packing Demo'}


In [37]:
feasible_sols=np.where(samplest.record.is_feasible==True)

In [38]:
feasible_sols[0]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78])